# Recommender System 

## Doelen


#### Algoritme 1: item-based filtering

Per stad willen we de beste bedrijven aanbevelen die voor de gebruikers het meest relevant zijn. Een recommandatie zal worden gedaan met behulp van item-based filtering zoals het aantal sterren dat een bedrijf gemiddeld krijgt. Er zal uiteindelijk een top tien van beste bedrijven worden aanbevolen.


#### Algoritme 2: content-based filtering

Content-based filtering wordt gebruikt om de verschillende soorten bedrijven te filteren. Dit filteren zal gebeuren op basis van 'categories' en deels user ratings. Het filteren van bedrijven kan op veel verschillende manieren gedaan worden. Wij hebben gekozen om het zo algemeen mogelijk te houden.


# Deel 1: data analyse

##### 1:  Laad de data in

In [1]:
# algemeen
import pandas as pd
import json

# voor het vinden van alle categorieën en attributes
import glob
import itertools

# import de items voor het recommender systeem zelf
from pandas import Series, DataFrame
import numpy as np

# voor de grafiek
import matplotlib.patches as mpatches

# for NLP
import spacy
nlp = spacy.load('en_core_web_md')

# voor aanbevelingen
import operator

def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

#### 2: voorbeeld van een stad in de data

In [2]:
data_path = './data/clarkson/'
review_file = 'review.json'
business_file = 'business.json'
user_file = 'user.json'
tip_file = 'tip.json'
checkin_file = 'checkin.json'

reviews = load_jsons(data_path, review_file)
businesses = load_jsons(data_path, business_file)
users = load_jsons(data_path, user_file)
tips = load_jsons(data_path, tip_file)
checkins = load_jsons(data_path, checkin_file)

In [3]:
print()
print("------------ Review Table ------------")
display(reviews.head())
print()
print("------------ Business Table ------------")
display(businesses.head())
print()
print("------------ User Table ------------")
display(users.head())
print()
print("------------ Tips Table ------------")
display(tips.head())
print()
print("------------ Check-in Table ------------")
display(checkins.head())


------------ Review Table ------------


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,H-4iwfPkxe5v5p2rtzUonQ,XuCbLgo9j1q5dDh9251vkg,AtOE7tSrJn1HXnU9YoUbPQ,2.0,2,2,2,"When interviewing for a job, they say the inte...",2011-02-25 16:49:13
1,7Q0tmeJv-IdtpIFl6bgSWg,fkMiipV4j_DG5nSoBnZvbg,AtOE7tSrJn1HXnU9YoUbPQ,3.0,1,0,1,I had a few minutes in my lunch hour to spare ...,2011-11-06 23:32:02
2,NGgax4OKfiNwjm_4hrqXGQ,MgYq6LXVz2-UFPLjFSl6jA,AtOE7tSrJn1HXnU9YoUbPQ,4.0,0,0,0,"Lunch...Fresh. Ordered a sandwich, fresh bread...",2017-02-03 18:07:31
3,Bmqoc7VC55p1CTywIt5hHA,cPd5M0RSDmHxrqf_WYPi4A,AtOE7tSrJn1HXnU9YoUbPQ,4.0,0,0,0,I work nearby and this place is a great option...,2018-01-19 21:08:46
4,E7IumF-cuw3C9XWGO7R7Qw,W5SoL5UhYBta28oZuhSjyQ,pRfL5hxA10iFCYEvEV8AQw,2.0,0,0,0,This is an okay place to go with the mates but...,2015-11-06 02:49:01



------------ Business Table ------------


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,AtOE7tSrJn1HXnU9YoUbPQ,Der Brotkorb the European Bakery Cafe,10 Woodchester Mall,Clarkson,ON,L5K 1R8,43.527340,-79.676598,3.0,4,1,"{'WiFi': 'u'no'', 'HasTV': 'False', 'BikeParki...",Restaurants,None
1,pRfL5hxA10iFCYEvEV8AQw,McDonald's,1829 Lakeshore Boulevard W,Clarkson,ON,L5J 1J6,43.516246,-79.625759,2.5,3,1,"{'NoiseLevel': 'u'average'', 'RestaurantsAttir...","Fast Food, Burgers, Restaurants","{'Monday': '6:0-0:0', 'Tuesday': '6:0-0:0', 'W..."
2,8HycRNzRmhwEkZiMD8X9kw,Fionn MacCool's Irish Pub,970 Southdown Road,Clarkson,ON,L5J 2Y4,43.510250,-79.630191,2.5,13,1,"{'RestaurantsReservations': 'True', 'BikeParki...","Beer, Wine & Spirits, British, Nightlife, Food...","{'Monday': '11:0-0:0', 'Tuesday': '11:0-0:0', ..."



------------ User Table ------------


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,gS4QCL6j_umAyfJXBGAZJQ,Maheen,400,2011-08-15 15:55:30,728,199,401,"2011,2012,2013,2014,2015,2016,2017,2018","SMn0noNhAb9_nzb5at4mIQ, qHwj63QRTu_rWWGMNuoQlg...",24,...,4,3,2,5,25,31,40,40,26,0
1,XuCbLgo9j1q5dDh9251vkg,Jonathan,241,2011-02-03 19:03:37,512,211,220,"2011,2012,2013,2014,2015,2016","Qaf3EUJCwJ1Jg8lfbsY6Dw, FCCLvXYIxPaRPWWJUzajeg...",11,...,2,1,0,0,29,30,34,34,14,2
2,Jb13QyXGi0zD4ZtSTBQwpQ,Alex,302,2013-10-10 11:14:49,388,182,192,"2016,2017,2018","t_m1a0KzG960Vh5wgHo9YQ, Yj3zqXhYcB1qKQrw51YPoQ...",22,...,1,0,1,0,11,11,6,6,14,3
3,fkMiipV4j_DG5nSoBnZvbg,Martin,553,2011-03-02 19:46:15,1339,324,885,"2012,2013,2014,2015,2016,2017,2018","7JpPvfDAE_14vA1xgFOFjw, Z_0phc_GF_oU9cHXuOGRgA...",109,...,9,2,2,0,136,320,251,251,92,107
4,NZfuLlP0I87gQGVWNFUFRA,Christina,220,2013-02-22 08:17:16,257,80,49,,"e3im4jT74L811sAYtW544A, IrpBDdqzLCuXPZTHWzcjIw...",5,...,1,0,1,0,3,7,2,2,0,0



------------ Tips Table ------------


,user_id,business_id,text,date,compliment_count
0,fkMiipV4j_DG5nSoBnZvbg,AtOE7tSrJn1HXnU9YoUbPQ,Mmmmm homemade mustard.,2012-08-10 17:06:17,0
1,fkMiipV4j_DG5nSoBnZvbg,AtOE7tSrJn1HXnU9YoUbPQ,Mmmmmm perogies. HOME MADE!!,2012-03-30 16:28:32,0
2,fkMiipV4j_DG5nSoBnZvbg,AtOE7tSrJn1HXnU9YoUbPQ,It's the soup!!!!!,2012-03-09 17:27:15,0
3,sya4uhCAC9cCakEUjyEfKw,8HycRNzRmhwEkZiMD8X9kw,Brand new menu!! Some amazing new additions!,2016-10-20 22:38:46,0



------------ Check-in Table ------------


,business_id,date
0,8HycRNzRmhwEkZiMD8X9kw,"2011-12-15 22:52:58, 2011-12-16 19:12:47, 2011..."
1,AtOE7tSrJn1HXnU9YoUbPQ,"2012-03-09 17:27:15, 2012-03-30 16:28:33, 2012..."
2,pRfL5hxA10iFCYEvEV8AQw,"2014-06-01 14:06:25, 2016-07-27 19:58:13, 2016..."


#### 3: category filtering 
##### 3.1: itereer over alle files in de ./data directory

Pak uit elke stad in ./data de business-file. Hieruit worden alle categorieën aan een algemene lijst toegevoegd.

In [4]:
# de files
files = glob.glob("./data/*/")

lists = []

# stop alle categorieën in een lijst
for data_path in files:
    businesses = load_jsons(data_path, 'business.json')
    lists.append(businesses.categories.values)
    if len(businesses) == 100:
        print(data_path)

./data/kannapolis/
./data/macedonia/


##### 3.2: split alle lijsten in 'lists' op de komma's 

Gebruik daarna itertools om alle aparte lijsten samen te voegen tot 1. Verwijder ook de None values. Door er op het laatst een set van te maken, worden meteen alle dubbele categorieën eruit gefilterd.

In [5]:
categories = set([x.strip() for x in set(itertools.chain.from_iterable([y.split(',') for x in lists for y in x if y]))])

In [6]:
# print(categories)
print(len(categories))

1300


#### 4: Attribute filtering 
##### 4.1: itereer over alle files in de ./data directory

Pak uit elke stad in ./data de business-file. Doordat attributes dictionaries zijn, pak alleen de keys wanneer de dict niet None is. Voeg hierna alle lijsten van attributes toe aan een algemene lijst.

In [7]:
# de files
files = glob.glob("./data/*/")

lists = []

# stop alle categorieën in een lijst
for data_path in files:
    businesses = load_jsons(data_path, 'business.json')
    for x in range(len(businesses.attributes.values)):
        if businesses.attributes.values[x]:
            lists.append(list(businesses.attributes.values[x].keys()))


##### 3.2: voeg alle kleinere lijsten uit de grote lijst samen

Maak er een set van om alle dubbele eruit te filteren

In [8]:
attributes = set(itertools.chain.from_iterable(lists))

# print(attributes)
print(len(attributes))

39


# Deel 2: De algoritmes

## Algoritme 1: item-based filtering

### Bedrijven en hun ratings

##### Utility matrix

Om een begin te kunnen maken aan het item-based algoritme, moet er eerst een tabel op worden gesteld die gebruiks hun ratings voor bedrijven laat zien. Dit kan door de "Review Table" te gebruiken:


In [9]:
print()
print("------------ Review Table ------------")
display(reviews)
print()


------------ Review Table ------------


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,H-4iwfPkxe5v5p2rtzUonQ,XuCbLgo9j1q5dDh9251vkg,AtOE7tSrJn1HXnU9YoUbPQ,2.0,2,2,2,"When interviewing for a job, they say the inte...",2011-02-25 16:49:13
1,7Q0tmeJv-IdtpIFl6bgSWg,fkMiipV4j_DG5nSoBnZvbg,AtOE7tSrJn1HXnU9YoUbPQ,3.0,1,0,1,I had a few minutes in my lunch hour to spare ...,2011-11-06 23:32:02
2,NGgax4OKfiNwjm_4hrqXGQ,MgYq6LXVz2-UFPLjFSl6jA,AtOE7tSrJn1HXnU9YoUbPQ,4.0,0,0,0,"Lunch...Fresh. Ordered a sandwich, fresh bread...",2017-02-03 18:07:31
3,Bmqoc7VC55p1CTywIt5hHA,cPd5M0RSDmHxrqf_WYPi4A,AtOE7tSrJn1HXnU9YoUbPQ,4.0,0,0,0,I work nearby and this place is a great option...,2018-01-19 21:08:46
4,E7IumF-cuw3C9XWGO7R7Qw,W5SoL5UhYBta28oZuhSjyQ,pRfL5hxA10iFCYEvEV8AQw,2.0,0,0,0,This is an okay place to go with the mates but...,2015-11-06 02:49:01
5,oACdDlpQoBZ2HlV8TSJPiw,W5SoL5UhYBta28oZuhSjyQ,pRfL5hxA10iFCYEvEV8AQw,5.0,0,0,0,"This McDonald's is really nice, like it is pim...",2015-07-27 16:07:23
6,enMhUMBC1fr6OJX5GT1v2w,FsxmSCzD4BGBK6PTIQvKug,pRfL5hxA10iFCYEvEV8AQw,1.0,0,0,0,"Hello fellow foodies , last night at 9pm I had...",2018-09-26 14:11:18
7,ONagKuSf2XrkE81T9NbpvQ,Jb13QyXGi0zD4ZtSTBQwpQ,pRfL5hxA10iFCYEvEV8AQw,4.0,1,1,1,"A lot better than it's given credit for, this ...",2017-08-14 00:11:33
8,EBipq_x0nNhcQPmCA5fVwg,NZfuLlP0I87gQGVWNFUFRA,8HycRNzRmhwEkZiMD8X9kw,5.0,4,1,1,I absolutely love this bar. No matter the loc...,2013-03-20 15:02:22
9,l3kaP9zeOg1kw4xknywj5A,ZxtU74SJMRoB8ZQXWwNtRQ,8HycRNzRmhwEkZiMD8X9kw,5.0,0,0,0,Delicious food. Diverse menu. Competitive pr...,2017-12-05 19:38:59


In [10]:
items = reviews[['user_id','business_id','stars']]

def get_rating(ratings, userId, movieId):
    lijst = ratings.loc[(ratings['user_id'] == userId) & (ratings['business_id'] == movieId), ['stars']].values
    if lijst:
        return lijst[0][0]
    else:
        return np.nan

def pivot_ratings(ratings):
    """ takes a rating table as input and computes the utility matrix """
    return ratings.pivot_table(index='business_id', columns='user_id', values='stars',  aggfunc='mean')

# test
utility_matrix = pivot_ratings(items)


utility_matrix.columns = [x for x in range(len(utility_matrix.columns))]
utility_matrix.index = [x for x in range(len(utility_matrix.index))]


## Plotten

#### Scatter Matrix
De scatter matrix laat per user en bedrijf zien welke users wel bedrijf een rating heeft gegeven. Voor de snelheid van het programma is al eerder een relatief kleine stad ingeladen. Deze stad heeft 3 bedrijven.

In [11]:
def plot_scatter_matrix(matrix, colors = ['#e41a1c', '#377eb8', '#4eae4b', '#994fa1', '#ff8101', '#000000', '#999999'], figsize = (8,8)):
    # setup plot
    max_val = matrix.max(axis = 1).max()
    min_val = matrix.min(axis = 1).min()
    fmatrix = matrix.fillna(min_val - 1)
    elements = fmatrix.shape[0]
    axarr = pd.plotting.scatter_matrix(fmatrix, s = 180.0, c = colors[:elements], figsize=figsize, alpha = 1.0, diagonal = '')
    m = fmatrix.shape[1]
    max_val = matrix.max(axis = 1).max()
    min_val = matrix.min(axis = 1).min()
    
    # set axes for all subplots
    for i in range(m):
        for j in range(m):
            axarr[i,j].set_xlim(min_val - 0.5, max_val + 0.5)
            axarr[i,j].set_ylim(min_val - 0.5, max_val + 0.5)

    # set labels for subplots
    labels = fmatrix.index 
    axarr[0,0].legend([mpatches.Circle((0,0),1,fc=c) for c in colors], labels, loc = 'lower left');

plot_scatter_matrix(utility_matrix)

## Similarity

#### Cosine Similarity
De cosine similarity gebruikt de voorgaande data om de similarity te berekenen tussen items.

In [12]:
def cosine_similarity(matrix, id1, id2):
    """Compute cosine similarity between two rows."""    

    # selecteer alleen rijen zonder NaN
    items = matrix.loc[id1].notna() & matrix.loc[id2].notna()

    if not items.any():
        return np.nan
    
    # Localiseer de id's in items
    rij1 = matrix.loc[id1][items]
    rij2 = matrix.loc[id2][items]

    # checks voor speciale situaties, anders doorgaan naar de formule
    # als ze gelijk zijn return 1
    if rij1.equals(rij2):
        return 1.0
    
    # als een van de twee rijen uit alleen maar 0-waardes bestaa return None
    check = [True if x == 0 else False for x in rij1.values ]
    if pd.Series(check).all() == True:
        return np.nan
    
    check = [True if x == 0 else False for x in rij2.values ]
    if pd.Series(check).all() == True:
        return np.nan
    
    # de teller
    teller= np.sum([rij1.values[x] * rij2.values[x] for x in range(len(rij1))])
  
    # de noemer
    noemer= (np.sqrt(np.sum([rij1.values[x]**2 for x in range(len(rij1))]))*
            np.sqrt(np.sum([rij2.values[x]**2 for x in range(len(rij1))])))
    
    return teller/noemer


def create_similarity_matrix_cosine(matrix):
    """creates the similarity matrix based on cosine distance"""
    
    # Maak de matrix en zet de index en colommen
    similarity_matrix = pd.DataFrame(0, index=matrix.index, columns=matrix.index, dtype=float)
    
    # Vul de rows met de juiste waardes berekend in de "cosine_similarity"-functie
    for y in matrix.index:
        for x in matrix.index:
            similarity_matrix.xs(y)[x] = cosine_similarity(matrix, x, y) 
     
    return similarity_matrix

similarity = create_similarity_matrix_cosine(utility_matrix)
display(similarity)

,0,1,2
0,1.0,NaN,NaN
1,NaN,1.0,NaN
2,NaN,NaN,1.0


## Evaluatie item-based filtering 

Het gebruik van user based similarity heeft weinig zin, door een groot gebrek aan reviews van users.

# Algoritme 2: filtering op eigenschappen

## Werking

#### 1: Category filtering
Ten eerste is er een lijst opgesteld van woorden waarop gegroepeerd kan worden. Er zitten, zoals ontdekt is in een bovenstaande cel, 1300 verschillende categorieën in de dataset. Door deze te groeperen in meer algemene groepen kan er een filtering plaatsvinden op basis van die categorieën.

#### 2: Similarity
Hierna wordt er een similarity matrix opgesteld, die zoekt naar de top 20 hoogste similarities in die set. Op basis van deze uitkomsten worden de meest relevante businesses gevonden.

###### voorbeeld: Food
De volgende lijst is opgesteld om alle categorieën die te maken hebben met de term "Food" samen te vatten:

{ 'Live/Raw Food', 'Bagels', 'Fish & Chips','Wraps', 'Steakhouses', 'Personal Chefs', 'Cheese Tasting Classes','Seafood', 'Buffets', 'Chocolatiers & Shops', 'Pub Food','Meat Shops', 'Gelato', 'Milkshake Bars','Food Tours', 'Shaved Ice','Fruits & Veggies', 'Desserts', 'Sandwiches', 'Hot Dogs', 'Food Delivery Services', 'Diners', 'Pizza', 'Chicken Wings','Pita',' Cheese Shops','Food Banks', 'Rotisserie Chicken’, 'Custom Cakes', 'Barbeque','Tapas Bars','Food Stands', 'Do-It-Yourself Food','Cupcakes', 'Bed & Breakfast', 'Pumpkin Patches', 'Local Fish Stores', 'Food Court',, 'Imported Food', 'Tacos', 'Comfort Food', 'Specialty Food', 'Sushi Bars', 'Vitamins & Supplements', 'Falafel','Sugaring', 'Honey', 'Seafood Markets', 'Cheesesteaks', 'Poke', 'Noodles', 'Sugar Shacks', 'Fast Food','Churros','Soul Food', 'Soup', 'Pretzels', 'Ice Cream & Frozen Yogurt','Bistros','Pasta Shops', 'International Grocery','Dim Sum', 'Food', 'Ramen', 'Patisserie/Cake Shop’, 'Bakeries', 'Breakfast & Brunch’, 'Bubble Tea', 'Restaurant Supplies', 'Eatertainment','Tasting Classes', 'Ethical Grocery’,  'Game Meat'}

Aan de hand van deze lijst kunnen items die geen van deze categorieën bevatten weg worden gehaald, omdat deze niet relevant zijn voor de gebruiker.

### Laad eerst een grotere dataset in

In [13]:
data_path = './data/sainte-anne-de-bellevue/'
review_file = 'review.json'
business_file = 'business.json'
user_file = 'user.json'
tip_file = 'tip.json'
checkin_file = 'checkin.json'

reviews = load_jsons(data_path, review_file)
businesses = load_jsons(data_path, business_file)
users = load_jsons(data_path, user_file)
tips = load_jsons(data_path, tip_file)
checkins = load_jsons(data_path, checkin_file)

## Filtering op categorie

In [14]:
# Maak de "food" lijst
food_items = [ 'Live/Raw Food','Restaurants' ,'Bagels', 'Fish & Chips','Wraps', 'Steakhouses', 'Personal Chefs', 
              'Cheese Tasting Classes','Seafood', 'Buffets', 'Chocolatiers & Shops', 'Pub Food','Meat Shops', 
              'Gelato', 'Milkshake Bars','Food Tours', 'Shaved Ice','Fruits & Veggies', 'Desserts', 'Sandwiches', 
              'Hot Dogs', 'Food Delivery Services', 'Diners', 'Pizza', 'Chicken Wings','Pita',' Cheese Shops',
              'Food Banks', 'Rotisserie Chicken', 'Custom Cakes', 'Barbeque','Tapas Bars','Food Stands', 
              'Do-It-Yourself Food','Cupcakes', 'Bed & Breakfast', 'Pumpkin Patches', 'Local Fish Stores', 
              'Food Court', 'Imported Food', 'Tacos', 'Comfort Food', 'Specialty Food', 'Sushi Bars', 
              'Vitamins & Supplements', 'Falafel','Sugaring', 'Honey', 'Seafood Markets', 'Cheesesteaks', 
              'Poke', 'Noodles', 'Sugar Shacks', 'Fast Food','Churros','Soul Food', 'Soup', 'Pretzels', 
              'Ice Cream & Frozen Yogurt','Bistros','Pasta Shops', 'International Grocery','Dim Sum', 'Food', 
              'Ramen', 'Patisserie/Cake Shop', 'Bakeries', 'Breakfast & Brunch', 'Bubble Tea', 
              'Restaurant Supplies', 'Eatertainment','Tasting Classes', 'Ethical Grocery', 'Game Meat']

#ga na voor de items in de business categories of tenminste 1 item in de lijst valt:

## filter de NaN's uit de categorieën, omdat deze niet vergelijkbaar zijn
filtered = businesses.dropna(subset=['categories'])
business_name_categories = filtered[['name','categories']]


# pak alle indexes van de businesses die wel onder de categorie "Food" vallen
indexes = []
teller = 0
for categories in business_name_categories.categories:

    checklist = []
    categories = categories.split(',')

    for item in categories:
        item = item.strip()

        if item in food_items:
            checklist.append(item)
    if len(checklist) >= 1:
        indexes.append(teller)
    teller +=1 

indexes = list(set(indexes))
business_name_categories.index = [x for x in range(len(business_name_categories))]

# alle bedrijven binnen de categorie
true_items = business_name_categories.loc[business_name_categories.index.isin(indexes)]
display(true_items)

,name,categories
0,Arcade Basile,"Amusement Parks, Restaurants, Active Life, Bre..."
1,Olé Tapas,"Food, International Grocery, Restaurants, Tapa..."
2,Il Lago,"Restaurants, Italian"
3,Café Twigs,"Restaurants, Cafes, French"
4,Marco Bar & Grill,"Restaurants, Pizza, Seafood, Italian"
5,Klondike,"Steakhouses, Restaurants, International, Italian"
6,Resto Pub Bord'Eaux,"Gastropubs, Restaurants, Nightlife, Canadian (..."
7,Bistro Boaboa,"Restaurants, Korean"
8,Tandoori Bellevue,"Indian, Restaurants"
9,Cunninghams,"Irish Pub, Bars, Restaurants, Nightlife"


,name,categories
11,Spa Concept Manon Lemay,"Day Spas, Reflexology, Hair Removal, Laser Hai..."
15,Montreal Printing,"Mass Media, Local Services, Printing Services,..."
18,Zoo Ecomuseum,"Museums, Active Life, Arts & Entertainment, Zoos"
19,Ville de Sainte-Anne-de-Bellevue,"Local Flavor, Public Services & Government, La..."
22,Annies Sur-Le-Lac,"Bars, Nightlife"


## Filtering op similarity

Maak een dataframe aan die elke business vergelijkt op categories

In [15]:
filtered = true_items.dropna(subset=['categories'])
items = filtered['categories'].values.tolist()
names = filtered.name.values

lijst = [nlp(x).similarity(nlp(y))  for x in items for y in items]

values = []
teller = 0
for x in range(len(items)):
    lijst2 = [lijst[teller:(teller+y)] for y in range(len(items)+1)]
    teller+=len(items)
    for x in lijst2:
        if len(x) == len(items):
            values.append(x)

df_nlp_similarity = pd.DataFrame(data = values,index=names, columns=names)
# display(df_nlp_similarity)

## Aanbevelen op basis van eerdere positieve waardering

#### Positieve gebruiker
Voor een gebruiker die een positieve rating heeft gegeven aan een bedrijf, geef de top 20 bedrijven terug die de hoogste similarity hebben. Bewaar vervolgens voor de lijst met gefilterde ratings alleen de user_id en naam van de business.

In [16]:
# maak een tabel met alleen de user_id, business_id en de stars
user_star_business = reviews[['user_id','business_id','stars']]

# een rating is positief als de waarde hoger is dan de gemiddelde rating van alle ratings
average_rating = user_star_business['stars'].mean()
user_star_business.loc[user_star_business['stars'] > average_rating]

# verander de business_id naar de daadwerkelijke naam van de business, en sla die naam en de user_id op
naming = businesses[['business_id','name']]
lijst = [naming.loc[naming['business_id'] == x , 'name'].values for x in [x for x in user_star_business['business_id']]]

user_star_business['company name'] = [x[0] for x in lijst]
business_user = user_star_business[['company name','user_id']]
business_user

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,company name,user_id
0,Arcade Basile,MYr_7OhQNfDW5W4EfrrIxQ
1,Olé Tapas,oTXacfmCLz9mev6KORIfEQ
2,Arcade Basile,DN8GeqNTMLrD5QSy2zPkCw
3,Olé Tapas,11TSdzlObGP51nyDgPxXaw
4,Olé Tapas,QDNbKtGO_I_m6lOUhVH9gw
...,...,...
209,Bellevue Resto-Grill,x459culD4Wy_auLcAoQgrw
210,Bellevue Resto-Grill,hO4etmRGC-en7UCuR-POzA
211,Bellevue Resto-Grill,gxKcF4vfQc8B_4reEtKXEA
212,Bellevue Resto-Grill,R0lfYlL4NbE8i8ZzW19muA


#### Voorbeeld user recommendation
Nu we een dataframe hebben met alle users die een bepaald bedrijf gerelateerd aan eten een hoger dan gemiddelde rating hebben gegeven, geven we een voorbeeld voor hoe een dergelijke aanbeveling werkt. Laten we uit gaan van de items met index 0 van de voorgaande tabel. Dit is het restaurant "Arcade Basile", gerate door user "MYr_7OhQNfDW5W4EfrrIxQ".

In [17]:
# pak ten eerste het bedrijf
company = business_user['company name'][0]

# pak alle bijbehorende waardes en similarities
companies = df_nlp_similarity.loc[df_nlp_similarity.index == company]

# pak vervolgens alle similarities
values = [x for x in df_nlp_similarity[company]]

# sorteer alle similarities en geef de top 20 items terug
company_similarity_dict = sorted( dict(zip(companies,values)).items(), key=operator.itemgetter(1), reverse= True)

recommended = [x[0] for x in company_similarity_dict[1:21]]
print(recommended)

["Au Bout de l'Isle", 'Olé Tapas', 'Bellevue Resto-Grill', 'Dundees Deli & Bar', "Resto Pub Bord'Eaux", 'Marco Bar & Grill', 'La Fondue du Prince', 'Klondike', 'Café Twigs', 'Cunninghams', 'Violet Angel', 'Surcouf', "Peter's Cape Cod", 'Il Lago', 'Tandoori Bellevue', 'Restaurant Chalet Thai', 'Bistro Boaboa', 'Stoolies Coffee House']


## Evaluatie content-based filtering

Zoals te zien is, zijn er 20 bedrijven aanbevolen, waarbij de similarity met de target business steeds lager wordt. Het is dus mogelijk om op basis van eigenschappen van de bedrijven aanbevelingen te doen.

# Deel 3: Evaluatie

Voor de evaluatie hebben wij besloten recall, precision en accuracy te gebruiken. Om dit te kunnen doen moeten we de categories van de aanbevolen bedrijven weten. Om dit te berekenen hebben wij de volgende items nodig:

- True positives: aanbevolen bedrijven die overeenkomende categorieën hebben met het target bedrijf
- True negatives: niet aanbevolen bedrijven die ook geen overeenkomende categorieën hebben met het target bedrijf
- False positives: aanbevolen bedrijven die geen overeenkomende categorieën hebben met het target bedrijf
- False negatives: niet aanbevolen bedrijven die ook overeenkomende categorieën hebben met het target bedrijf

In [19]:
# pak alle categorieën van alle aanbevolen bedrijven
categories = [businesses.loc[businesses['name'] == x, 'categories'].values for x in recommended]

evaluation_table = pd.DataFrame({'name':recommended,'categories':categories})
display(evaluation_table)

# alle bedrijven buiten de categorie
false_items = business_name_categories.loc[~business_name_categories.index.isin(indexes)]
display(false_items)

,name,categories
0,Au Bout de l'Isle,"[Soup, Breakfast & Brunch, Sandwiches, Restaur..."
1,Olé Tapas,"[Food, International Grocery, Restaurants, Tap..."
2,Bellevue Resto-Grill,"[Seafood, Bars, Mediterranean, Nightlife, Rest..."
3,Dundees Deli & Bar,"[Bars, Nightlife, Restaurants, Delis, Sandwich..."
4,Resto Pub Bord'Eaux,"[Gastropubs, Restaurants, Nightlife, Canadian ..."
5,Marco Bar & Grill,"[Restaurants, Pizza, Seafood, Italian]"
6,La Fondue du Prince,"[Restaurants, Fondue, Barbeque, Chinese, French]"
7,Klondike,"[Steakhouses, Restaurants, International, Ital..."
8,Café Twigs,"[Restaurants, Cafes, French]"
9,Cunninghams,"[Irish Pub, Bars, Restaurants, Nightlife]"


,name,categories
11,Spa Concept Manon Lemay,"Day Spas, Reflexology, Hair Removal, Laser Hai..."
15,Montreal Printing,"Mass Media, Local Services, Printing Services,..."
18,Zoo Ecomuseum,"Museums, Active Life, Arts & Entertainment, Zoos"
19,Ville de Sainte-Anne-de-Bellevue,"Local Flavor, Public Services & Government, La..."
22,Annies Sur-Le-Lac,"Bars, Nightlife"


Nu we een dataframe hebben met alle businesses en hun categories, kunnen we de precision, recall en accuracy uitrekenen met de volgende formules:

- precision = true positives / true positives + false positives
- recall = true positives / true positives + false negatives
- accuracy = (True Positives + True Negatives) /(True Positives + True Negatives + False Positives + False Negatives)

In [60]:

# print(company)
# for i, row in businesses.dropna().iterrows():
#     bag = []
#     k = row['categories']
#     for j, row2 in businesses.dropna().iterrows():
#         p = row2['categories']
#         if k != p:
#             bag.append([row2['name'], (nlp(k).similarity(nlp(p)))])
            
#     df = pd.DataFrame(bag, columns =['Name', 'similarity'], dtype = float) 

# display(df.nlargest(20, 'similarity'))